In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()

driver.get('https://www.myntra.com/dresses?f=Gender%3Amen%20women%2Cwomen&sort=popularity')

In [11]:
title = driver.title
print(title)

Dresses Online - Low Price Offer on Dresses for Women at Myntra


In [12]:
item = driver.find_element(by=By.ID, value='25892106')

In [13]:
def extract_product_info(li_element):
    try:
        # Locate the product metadata container
        product_meta_info = li_element.find_element(By.CLASS_NAME, "product-productMetaInfo")

        # Extract the product id
        product_id = li_element.get_attribute("id")

        # Extract the brand
        brand = product_meta_info.find_element(By.CLASS_NAME, "product-brand").text

        # Extract the product name
        product_name = product_meta_info.find_element(By.CLASS_NAME, "product-product").text

        # Extract the price
        price = _extract_product_price(product_meta_info)

        # Locate the image slider container
        image_slider = li_element.find_element(By.CLASS_NAME, "product-imageSliderContainer")

        # Extract the <img> tag inside the <picture> tag
        img_tag = image_slider.find_element(By.TAG_NAME, "img")
        image_url = img_tag.get_attribute("src")

        # Extract the product link
        product_link = li_element.find_element(By.TAG_NAME, "a").get_attribute("href")

        # Extract the product rating
        rating, count = _extract_rating_data(li_element)

        return {
            "id": product_id,
            "brand": brand,
            "product_name": product_name,
            "price": price,
            "rating:": rating,
            "count": count,
            "image_url": image_url,
            "product_link": product_link,
        }

    except Exception as e:
        print(f"Error extracting product info from <li>: {e}")
        return None



def _extract_product_price(product_meta_info):
    price_container = product_meta_info.find_element(By.CLASS_NAME, "product-price")

    # Check if discounted price is present
    price = {"Discounted": 0, "Original": 0}
    try:
        discounted_price = price_container.find_element(By.CLASS_NAME, "product-discountedPrice").text.replace("Rs. ", "").strip()
        original_price = price_container.find_element(By.CLASS_NAME, "product-strike").text.replace("Rs. ", "").strip()
        # price = f"Discounted: {discounted_price}, Original: {original_price}"
        price["Discounted"]= discounted_price
        price["Original"]= original_price

    except:
        # If discounted price is not present, fall back to the single price
        original_price = price_container.find_element(By.TAG_NAME, "span").text.replace("Rs. ", "").strip()
        price["Original"] = original_price

    return price


def _extract_rating_data(li_element):
    try:
        # Locate the ratings container
        ratings_container = li_element.find_element(By.CLASS_NAME, "product-ratingsContainer")

        # Extract the rating value (e.g., 4)
        rating_value = ratings_container.find_element(By.TAG_NAME, "span").text

        # Extract the ratings count (e.g., 872)
        ratings_count_container = ratings_container.find_element(By.CLASS_NAME, "product-ratingsCount")
        ratings_count = ratings_count_container.text.split('|')[-1].strip()

        return rating_value, ratings_count

    except Exception as e:
        print(f"Error extracting data from an <li>: {e}")
        return None, None